<a href="https://colab.research.google.com/github/adititadkod15-tech/HinglishLID1/blob/main/Qwen2_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from transformers import pipeline

In [3]:
# 1. Setup
model_id = "Qwen/Qwen2.5-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [52]:
system_prompt = """Act as a strict TOKEN-FILTER. Your only job is to extract and transliterate Roman-script Hindi words.

### OPERATION RULES:
1. DICTIONARY CHECK: Before processing a word, check if it is a standard English dictionary word (e.g., 'directly', 'heartache', 'politics', 'war').
2. PROPER NOUN CHECK: Identify names of people, places, and specific events (e.g., 'Cambodian', 'India', 'Civil War').
3. ACTION: If a word passes Rule 1 or Rule 2, DELETE it. Do not process it.
4. TRANSLITERATION ONLY: For remaining Hindi words, provide phonetic transliteration into Devanagari.
   - STRICT: Never translate (e.g., do not turn 'directly' into 'दर्शनीय').
   - STRICT: Never transliterate English (e.g., do not turn 'War' into 'वॉर').

### OUTPUT FORMAT:
[original_word] -> [devanagari]

### EXCLUSION EXAMPLES (Do NOT list these):
- English: interviewee, chemical, weapons, dangerous, during, heartache, dismay, melancholic, roundtable, discussion, emotional, directly.
- Proper Nouns: Cambodian, Civil War, India, Naxalite, Maoist, Eritrean, Fijian Coups."""


In [55]:
def get_transliterated_hinglish(sentence):
    messages = [
        {"role": "system", "content": system_prompt},
        # High-impact negative example
        {"role": "user", "content": "Sentence: Politics ke peeche war hota hai aur directly nuksan hota hai."},
        {"role": "assistant", "content": "ke -> के\npeeche -> पीछे\nhota -> होता\nhai -> है\naur -> और\nnuksan -> नुकसान\nhota -> होता\nhai -> है"},
        {"role": "user", "content": f"Sentence: {sentence}"}
    ]

    prompt = pipe.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    outputs = pipe(
        prompt,
        max_new_tokens=750, # Lower tokens reduce the chance of 'chatty' hallucination
        do_sample=False,
        tokenizer=pipe.tokenizer,
        stop_strings=["<|im_end|>", "<|endoftext|>"]
    )

    return outputs[0]["generated_text"].strip()

In [56]:
sample_sentences =[
    'Pahle toh hum baat karte hain Cambodian Civil War ki (999 words). Wahan par jo hua, woh sach mein heartache aur shok se bhara tha. Lakhon logon ki zindagiyan barbad ho gayihi. Kitni hi auraten vidhvaa ho gayihi, aur kitne bachche anath! Sab ke chehron par sirf nirasha aur melancholic hi nazar aata hoga us samay.',
    'Mujhe yaad hai jab maine ek documentary dekha tha is war ke baare mein. Woh sab sunkar mera rooh kaanp gaya tha. Bachchon ko apne maa-baap ko khote dekhna, auraton ki vidhvavastha, ye sab dekhkar ekdum se mera man udaas aur dismay se bhar gaya tha. ',
    'Jaisa ki aap sab jaante hain, wars ke beech chemical aur biological weapons ka istemaal bhi hota hai. Inse toh aur bhi nuksan hota hai zindagiyon ka. Ek missile ya bomb se lakhon log dukhi ho jaate hain - apne ghar gawaa dete hain, apne parivaar ko khaa dete hain. Ye sab sochna hi itna hidaayatheen karta hai',
    'Aaj ki is roundtable discussion ka mukhy vishay bahut hi bhavuk hai (999 words) - wars aur conflicts ke emotional asar par charcha karni hai. Ye ek aisi cheez hai jo har insaan ko chhoo kar jaati hai, chahe woh usmein directly shaamil ho ya na ho.',
    'Fijian Coups ke baare mein padha tha maine kuch dino pehle. Wahan par bhi bahut kuchh hua tha. Aakhir kyun hoti hain ye sab cheezein? Kyun koi use bombs aur weapons karta hai doosre ki zindagiyon ko barbad karne ke liye? Ye sab jaanne ke baad toh main ekdum hi nimhans ho gaya tha.',
    'Aur jaisa ki hum jaante hain, India bhi aise hi kayi wars aur insurgencies se guzra hai. Jaise ki Naxalite-Maoist insurgency, aur Eritrean War of Independence. In dono mein kitne hi jawaan shaheed hue, aur kitne gareeb logon ko apni zindagiyaan gawani padin. Sab ke ghar barbad ho gaye, sari umeeden toot gayihi.',
    'War politics ke peeche badi wajah ye fear of war hota hai har country ka (999 words). Har koi chahta hai ki unhe jeetna hai, unhe power mein rehna hai. Issi liye ye sab wars shuru hote rehte hain. Par in sab ke peeche, sabse bada nuksan toh aam insaanon ka hi hota hai. ',

]

for sample_sentence in sample_sentences:
  result = get_transliterated_hinglish(sample_sentence)
  print(result)

<|im_start|>system
Act as a strict TOKEN-FILTER. Your only job is to extract and transliterate Roman-script Hindi words.

### OPERATION RULES:
1. DICTIONARY CHECK: Before processing a word, check if it is a standard English dictionary word (e.g., 'directly', 'heartache', 'politics', 'war').
2. PROPER NOUN CHECK: Identify names of people, places, and specific events (e.g., 'Cambodian', 'India', 'Civil War').
3. ACTION: If a word passes Rule 1 or Rule 2, DELETE it. Do not process it.
4. TRANSLITERATION ONLY: For remaining Hindi words, provide phonetic transliteration into Devanagari. 
   - STRICT: Never translate (e.g., do not turn 'directly' into 'दर्शनीय'). 
   - STRICT: Never transliterate English (e.g., do not turn 'War' into 'वॉर').

### OUTPUT FORMAT:
[original_word] -> [devanagari]

### EXCLUSION EXAMPLES (Do NOT list these):
- English: interviewee, chemical, weapons, dangerous, during, heartache, dismay, melancholic, roundtable, discussion, emotional, directly.
- Proper Nouns: Ca